In this work I am going to build an ensemble of classifiers to see how sentiment scores taken on text related to ethnic issues can be predicted.

In [14]:
#firts of all, loading the data with a text column and a score column
import pandas as pd
import numpy as np
df = pd.read_excel('etnic_example.xlsx', index_col = None)

df = df[['source', 'text']]
num_rows, num_feature =df.shape
print('full data loaded')
print('number of rows: ', num_rows)
print('number of columns (features): ', num_feature)
print(list(df))
#as you can see, there are almost 4k of observations available

full data loaded
number of rows:  3833
number of columns (features):  2
['source', 'text']


In [15]:
df.head(5) #let's take a look what is inside of the data

,source,text
0,-1,"Гражданским дали коридор и сказали, что город ..."
1,-1,"Какой шикарный ублюдок) Сашок, еще что-нибудь ..."
2,-1,"[[id282525992|Проверочная], <b>чурки</b> это т..."
3,-1,"[[id12512286|Владимир], не-а. Я вообще не пони..."
4,-1,я не хочу верить или не верить..это не религия...


In [16]:
#further, cleaning and tokenization of the data are done
import pymorphy2
import nltk
from nltk.tokenize import word_tokenize
morph = pymorphy2.MorphAnalyzer()

nltk.download('punkt')
import re

def clear_data(df):
    df['text'] = df['text'].apply(lambda text: text.strip().split('|')[-1].split('http')[0].strip().lower())
    df['text'] = df['text'].apply(lambda text: re.sub('[\?/~!:|)(«»@#$_12]34567890#—ツ►,.๑۩۞۩•*”˜˜”*°°*`&<>]', '', text))
    return df



def tokenize_and_stem(df):
    df['text'] = df['text'].apply(lambda text: word_tokenize(text))
    df['text'] = df['text'].apply(lambda text: [morph.parse(word)[0].normal_form for word in text])
    df['text'] = df['text'].apply(lambda text: ' '.join(text))
    return df


clean0 = clear_data(df)
clean = tokenize_and_stem(clean0)
clean.head()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,source,text
0,-1,"гражданский дать коридор и сказать , что город..."
1,-1,"какой шикарный ублюдок ) сашок , ещё что-нибуд..."
2,-1,"проверочный ] , < b > чурка < /b > это таджик ..."
3,-1,"владимир ] , не- . я вообще не понимать , что ..."
4,-1,я не хотеть верить или не верить..этый не рели...


In [17]:
#then, I am creating a vector of words from cleaned and toknized data
words = []


for line in open('stopwords_rus.txt'):
    line = line.split('\n') 
    line = line[0]
    words.append(line)

words = frozenset(words)

print('Vectorization of documents')
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer(ngram_range=(1,1), stop_words=(words))

yx = vec.fit_transform(clean['text'])
ethno_data = vec.fit_transform(clean['text']).toarray()

num_docs, num_feature = yx.shape
print('documents number : ', num_docs, 'feature number: ', num_feature)

Vectorization of documents


C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aren', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'let', 'll', 'mustn', 'ours', 're', 'shan', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


documents number :  3833 feature number:  23174


In [18]:
#creating a vector of answers
Y = clean['source'].values
YY = Y.ravel()

#here, I split the data on tarining and test sets with a threshold of 0.2
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(ethno_data, YY, test_size=0.2, random_state = 27)

print('Test collection size: ', X_test.shape) #now, in the test data I have 767 observations
print('Training collection size: ', X_train.shape) #and in the train data there are 3066 observations

Test collection size:  (767, 23174)
Training collection size:  (3066, 23174)


In [19]:
#here, the ensembling starts
#I do not specify any hyperparameters for the models used as parts of the ensemble relying on auto settings
#believe me or not, with specified hyperparameters the ensemble performs even worse in terms of prediction power..

#here, I'll see how the ensemble performs on the train data
from sklearn import metrics
from sklearn import model_selection
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score


kfold = model_selection.KFold(n_splits=5, random_state=None)
# create the sub models
estimators = []
model1 = LogisticRegression()
estimators.append(('logistic', model1))

model2 = RandomForestClassifier()

estimators.append(('rf', model2))

model3 = SVC()
estimators.append(('svm', model3))

model4 = AdaBoostClassifier()
estimators.append(('adabust', model4))


ensemble = VotingClassifier(estimators, voting='hard')

results = model_selection.cross_val_score(ensemble, X_train, y_train, cv=kfold, scoring='f1_macro')
print(results.max())

C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Asus\Anaco

0.46058154217615094


The F1 score is waaay too small, that is a bad model, indeed. 

In [20]:
#in this chunk, I fit the model and get the F1 score for the test data

ensemble.fit(X_train, y_train)

scores = cross_val_score(ensemble, X_test, y_test, cv=5, scoring='f1_macro')
print(np.max(scores))

C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Asus\Anaco

0.39847189847189846


C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


And the score on the test data is even smaller :)

Looks like ensemble modeling failed. 